### Calculate the Area on the scalp that is covered by a probe. Can be used to assess the density of a probe: (# Optodes / Covered Area)

In [ ]:
import numpy as np
import xarray as xr
import cedalion.datasets
import cedalion.plots
import matplotlib.pyplot as p
import pickle
import os
import pandas as pd
import cedalion.imagereco.forward_model as fw
from cedalion.imagereco.solver import pseudo_inverse_stacked
import cedalion.geometry.landmarks as cd_landmarks
import pyvista as pv
import cedalion.dataclasses as cdc
pv.set_jupyter_backend('server')
import warnings
warnings.filterwarnings("ignore")
import configs

In [ ]:
datasets_path = configs.data_path_prefix

Load Headmodel

In [ ]:
SEG_DATADIR_cl27, mask_files_cl27, landmarks_file_cl27 = cedalion.datasets.get_colin27_segmentation()
colin_parcel_file = cedalion.datasets.get_colin27_parcel_file()

head_colin = fw.TwoSurfaceHeadModel.from_surfaces(
    segmentation_dir=SEG_DATADIR_cl27,
    mask_files = mask_files_cl27,
    brain_surface_file= os.path.join(SEG_DATADIR_cl27, "mask_brain.obj"),
    scalp_surface_file= os.path.join(SEG_DATADIR_cl27, "mask_scalp.obj"),
    landmarks_ras_file=landmarks_file_cl27,
    brain_face_count=None,
    scalp_face_count=None,
    parcel_file=colin_parcel_file,
    fill_holes=True
)

head_colin.brain.units = cedalion.units.mm
head_colin.scalp.units = cedalion.units.mm
head_colin.landmarks = head_colin.landmarks.pint.dequantify()
head_colin.landmarks.pint.units = cedalion.units.mm

lmbuilder = cd_landmarks.LandmarksBuilder1010(head_colin.scalp, head_colin.landmarks)
all_landmarks = lmbuilder.build()
head_colin.landmarks = all_landmarks

n_scalp = head_colin.scalp.nvertices
parcels_colin = np.concatenate(
    (head_colin.brain.vertices.coords["parcel"].values, n_scalp * ['scalp']))  # noqa: E501d


# Load ICBM152 headmodel

SEG_DATADIR_icbm, mask_files_icbm, landmarks_file_icbm = cedalion.datasets.get_icbm152_segmentation()
icbm_parcel_file = cedalion.datasets.get_icbm152_parcel_file()
head_icbm = fw.TwoSurfaceHeadModel.from_surfaces(
    segmentation_dir=SEG_DATADIR_icbm,
    mask_files = mask_files_icbm,
    brain_surface_file= os.path.join(SEG_DATADIR_icbm, "mask_brain.obj"),
    scalp_surface_file= os.path.join(SEG_DATADIR_icbm, "mask_scalp.obj"),
    landmarks_ras_file=landmarks_file_icbm,
    brain_face_count=None,
    scalp_face_count=None,
    parcel_file=icbm_parcel_file,
    fill_holes=True
)

head_icbm.brain.units = cedalion.units.mm
head_icbm.scalp.units = cedalion.units.mm
head_icbm.landmarks = head_icbm.landmarks.pint.dequantify()
head_icbm.landmarks.pint.units = cedalion.units.mm

lmbuilder = cd_landmarks.LandmarksBuilder1010(head_icbm.scalp, head_icbm.landmarks)
all_landmarks = lmbuilder.build()
head_icbm.landmarks = all_landmarks

n_scalp = head_icbm.scalp.nvertices
parcels_icbm = np.concatenate(
    (head_icbm.brain.vertices.coords["parcel"].values, n_scalp * ['scalp']))  # noqa: E501d

In [ ]:
use_icbm = True

# Load matrices

if use_icbm:
    head = head_icbm
    Adot_ninja = cedalion.datasets.get_precomputed_sensitivity('nn22_resting', head_model='icbm152')
    Adot_ninja = Adot_ninja.assign_coords(parcel = ("vertex", parcels_icbm))

    with open(datasets_path + 'HD_Squeezing/Adot/Adot_HDSqueezing_ICBM.pickle', 'rb') as f:
        Adot_HD_Squeezing = pickle.load(f)
    Adot_HD_Squeezing = Adot_HD_Squeezing.assign_coords(parcel = ("vertex", parcels_icbm))

else:
    head = head_colin
    # ninjanirs (resting state + synthetic)
    Adot_ninja = cedalion.datasets.get_precomputed_sensitivity('nn22_resting', head_model='colin27')
    Adot_ninja = Adot_ninja.assign_coords(parcel = ("vertex", parcels_colin))

    # HD Squeezing
    with open(datasets_path + 'HD_Squeezing/Adot/Adot_HD_Squeezing_Colin.pkl', 'rb') as f:
        Adot_HD_Squeezing = pickle.load(f)
    Adot_HD_Squeezing = Adot_HD_Squeezing.assign_coords(parcel = ("vertex", parcels_colin))


In [ ]:
data_type = 'HD_Squeezing'
#data_type = 'Syn_FT'

In [ ]:
if data_type == 'HD_Squeezing':
    rec = cedalion.datasets.get_fingertappingDOT()
    #rec = cedalion.io.read_snirf(datasets_path +  "HD_Squeezing/HD_Ballsqueezing/sub-170/nirs/sub-170_task-BallSqueezing_run-1_nirs.snirf")[0]
    Adot = Adot_HD_Squeezing
if data_type.startswith('Syn'):
    rec = cedalion.datasets.get_nn22_resting_state()
    #if data_type == 'Syn_Stroop':
    #    Adot = Adot_ninja_colin_stroop
    #if data_type == 'Syn_FT':
    #    Adot = Adot_ninja_colin_ft
    Adot = Adot_ninja
amp = rec["amp"]
geo3d = rec.geo3d
geo3d_snapped = head.align_and_snap_to_scalp(geo3d)
geo3d_snapped = geo3d_snapped[geo3d_snapped.type != cdc.PointType.LANDMARK]

if data_type.startswith('Syn'):
    with open(datasets_path + 'NN22_Resting_State/' + 'geo3d_snapped_ninja_colin.pkl', 'rb') as f:
        geo3d_snapped = pickle.load(f)

In [ ]:
geo3d_snapped = geo3d_snapped.sel(label=[l for l in geo3d_snapped.label.values if (l in amp.sel(channel=Adot.channel).source.values or l in amp.sel(channel=Adot.channel).detector.values)])

In [ ]:
cedalion.plots.plot_montage3D(rec["amp"], geo3d_snapped)

In [ ]:
amp

In [ ]:
ts_long, ts_short = cedalion.nirs.split_long_short_channels(
    amp, rec.geo3d, distance_threshold=2.2 * cedalion.units.cm
)

In [ ]:
plt = pv.Plotter()
#cedalion.plots.plot_surface(plt, head.brain, color="w")
cedalion.plots.plot_surface(plt, head.scalp)
cedalion.plots.plot_labeled_points(plt, geo3d_snapped, show_labels=True)
# save fig as pdf and svg
#if save_plot:
#    plt.show()
#    plt.save_graphic(os.path.join(base_path, "probe.pdf"))
#    plt.save_graphic(os.path.join(base_path, "probe.svg"))
#else:
plt.show()

In [ ]:
corner_optodes = [30, 32, 43, 45]

In [ ]:
plt = pv.Plotter()
#cedalion.plots.plot_surface(plt, head.brain, color="w")
cedalion.plots.plot_surface(plt, head.scalp)
cedalion.plots.plot_labeled_points(plt, geo3d_snapped[corner_optodes])
# save fig as pdf and svg
#if save_plot:
#    plt.show()
#    plt.save_graphic(os.path.join(base_path, "probe.pdf"))
#    plt.save_graphic(os.path.join(base_path, "probe.svg"))
#else:
plt.show()

In [ ]:
print(np.linalg.norm((geo3d_snapped[30] - geo3d_snapped[32])))
print(np.linalg.norm((geo3d_snapped[43] - geo3d_snapped[45])))
print(np.linalg.norm((geo3d_snapped[30] - geo3d_snapped[43])))
print(np.linalg.norm((geo3d_snapped[32] - geo3d_snapped[45])))

In [ ]:
print(np.linalg.norm((geo3d_snapped[43] - geo3d_snapped[44])))
print(np.linalg.norm((geo3d_snapped[44] - geo3d_snapped[45])))

In [ ]:
((80 * 67) / 100) * 2

In [ ]:
import cedalion.vis.plot_sensitivity_matrix
plotter = cedalion.vis.plot_sensitivity_matrix.Main(
    sensitivity=Adot,
    brain_surface=head.brain,
    head_surface=head.scalp,
    labeled_points=geo3d_snapped,
)
plotter.plot(high_th=3, low_th=2.99)

#if save_plot:
#    plotter.plt.save_graphic(os.path.join(base_path, "probe_sensitivity.pdf"))
#    plotter.plt.save_graphic(os.path.join(base_path, "probe_sensitivity.svg"))
plotter.plt.show()

In [ ]:
Adot_scalp = Adot.where(
        Adot["is_brain"] == False, drop=True
).sel(wavelength = 760).sum(dim='channel')

Adot_scalp[Adot_scalp <= 0] = Adot_scalp[
    Adot_scalp > 0
].min()

In [ ]:
sensivity_faces = Adot_scalp.values[np.array(head.scalp.mesh.faces)].min(axis=1)
active_faces = np.where(sensivity_faces > 3)[0]
print("Active faces: " + str(active_faces.size) + " (= " + str(active_faces.size * 100 / head.scalp.mesh.faces.shape[0]) + "%)")
active_area = head.scalp.mesh.area_faces[active_faces].sum() / 100
print("Active area: ", active_area)
geo3d_snapped = geo3d_snapped[geo3d_snapped.type != cdc.PointType.LANDMARK]
print("Number of optodes: ", geo3d_snapped.label.size)
print("Density Rate", geo3d_snapped.label.size / active_area)

In [ ]:
probe_area_data = {"active_area": active_area, "n_optodes": geo3d_snapped.label.size, "optodes_per_cm2": geo3d_snapped.label.size / active_area, 
                   "n_channels": amp.channel.size, "channels_per_cm2": amp.channel.size / active_area
}

In [ ]:
probe_area_data

In [ ]:
if data_type == 'HD_Squeezing':
    suffix = data_type
if data_type == 'Stroop_J':
    suffix = 'Stroop_Jessie'
if data_type == 'Stroop_D':
    suffix = 'Stroop_Deja'
if data_type.startswith('Syn'):
    suffix = 'NN22_Resting_State'

In [ ]:
write_path = os.path.join(datasets_path, suffix, "probe_area_data")
if data_type == 'Syn_FT':
    write_path += '_ft'

In [ ]:
#with open(write_path, 'wb') as f:
#    pickle.dump(probe_area_data, f)